# Polytope Basics: Introduction to Polytope Visualization

This notebook provides a comprehensive introduction to polytope visualization, including:
- Basic Platonic solids visualization
- Interactive polytope rendering with pythreejs
- Spherical harmonic analysis on polytopes
- Stereographic projections
- Biological applications (bacteria as deformed polytopes)
- Polytope tilings and patterns

## 1. Setup and Imports

In [ ]:
# Standard imports
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, HTML

# Set up matplotlib for notebook
plt.style.use('default')
%matplotlib inline

# Import our polytope modules
import sys
sys.path.append('..')

from src.polytope_core.platonic_solids import (
    create_all_platonic_solids,
    create_tetrahedron,
    create_cube,
    create_octahedron,
    create_icosahedron,
    create_dodecahedron
)

from src.visualization.polytope_viz import (
    PolytopeRenderer,
    PolytopeVisualization,
    create_platonic_visualizations,
    visualize_bacteria_deformation,
    StereographicProjectionViewer,
    PolytopeSignatureViewer,
    create_tiling_visualization
)

from src.visualization.harmonic_viz import (
    SphericalHarmonicVisualizer,
    BiologicalHarmonicPatterns,
    create_plotly_3d_harmonic
)

print("All modules loaded successfully!")

## 2. Basic Platonic Solids

### 2.1 Creating and Visualizing the Five Platonic Solids

The Platonic solids are the only convex polyhedra with congruent regular polygonal faces:
- **Tetrahedron**: 4 triangular faces
- **Cube**: 6 square faces
- **Octahedron**: 8 triangular faces
- **Dodecahedron**: 12 pentagonal faces
- **Icosahedron**: 20 triangular faces

In [ ]:
# Create all Platonic solids
platonic_solids = create_all_platonic_solids()

# Display their properties
for name, solid in platonic_solids.items():
    n_vertices = len(solid.vertices)
    n_edges = len(solid.edges) if solid.edges.size > 0 else 0
    n_faces = len(solid.faces) if solid.faces else 0
    
    print(f"\n{name.upper()}:")
    print(f"  Vertices: {n_vertices}")
    print(f"  Edges: {n_edges}")
    print(f"  Faces: {n_faces}")
    print(f"  Symmetry group: {solid.symmetry_group}")
    print(f"  Dual: {solid.dual_name}")

### 2.2 Static Matplotlib Visualization

In [ ]:
# Create a figure showing all Platonic solids
fig = plt.figure(figsize=(15, 3))

solid_names = ['tetrahedron', 'cube', 'octahedron']
colors = ['red', 'blue', 'green']

for idx, (name, color) in enumerate(zip(solid_names, colors)):
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    
    solid = platonic_solids[name]
    vertices = np.array(solid.vertices)
    
    # Plot vertices
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
               c=color, s=100, alpha=0.8, edgecolors='k')
    
    # Plot edges if available
    if solid.edges.size > 0:
        edges = np.array(solid.edges)
        for edge in edges:
            points = vertices[edge]
            ax.plot(points[:, 0], points[:, 1], points[:, 2], 
                   'k-', alpha=0.6, linewidth=2)
    
    # Set labels and appearance
    ax.set_title(name.capitalize(), fontsize=14, weight='bold')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_box_aspect([1,1,1])
    
    # Remove grid for cleaner look
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

plt.tight_layout()
plt.show()

## 3. Interactive 3D Visualization with pythreejs

### 3.1 Basic Interactive Renderer

In [ ]:
# Create interactive renderer
renderer = PolytopeRenderer(width=800, height=600)

# Get visualization objects
vis_polytopes = create_platonic_visualizations()

# Add tetrahedron and octahedron
tetrahedron = vis_polytopes['tetrahedron']
octahedron = vis_polytopes['octahedron']

# Position them side by side
tetrahedron_offset = tetrahedron._replace(
    vertices=tetrahedron.vertices + jnp.array([-2, 0, 0])
)
octahedron_offset = octahedron._replace(
    vertices=octahedron.vertices + jnp.array([2, 0, 0])
)

# Add to renderer
renderer.add_polytope(tetrahedron_offset, wireframe=True, solid=True, opacity=0.7)
renderer.add_polytope(octahedron_offset, wireframe=True, solid=True, opacity=0.7)

# Display
print("Use mouse to rotate, scroll to zoom:")
renderer.render()

## 4. Spherical Harmonic Analysis

### 4.1 Visualizing Spherical Harmonics on Polytopes

Spherical harmonics are the angular portion of the solution to Laplace's equation in spherical coordinates. They form an orthogonal basis for functions on the sphere and are fundamental in:
- Quantum mechanics (atomic orbitals)
- Shape analysis
- Signal processing on the sphere
- Biological shape characterization

In [ ]:
# Create spherical harmonic visualizer
harmonic_viz = SphericalHarmonicVisualizer(max_l=6)

# Use icosahedron vertices for high sampling
icosahedron = platonic_solids['icosahedron']
vertices = np.array(icosahedron.vertices)

# Visualize some key harmonics
fig = plt.figure(figsize=(15, 10))

# Define harmonics to visualize with biological interpretations
harmonics = [
    (0, 0, "Uniform (cell volume)"),
    (1, 0, "Dipole (cell polarity)"),
    (2, 0, "Quadrupole (elongation)"),
    (2, 1, "Oblique elongation (bending)"),
    (3, 0, "Octupole (branching)"),
    (4, 0, "Hexadecapole (complex symmetry)")
]

for idx, (l, m, description) in enumerate(harmonics):
    ax = fig.add_subplot(2, 3, idx+1, projection='3d')
    
    # Compute harmonic
    Y_lm = harmonic_viz.compute_spherical_harmonic(l, m, vertices)
    values = np.real(Y_lm)
    
    # Normalize for coloring
    vmin, vmax = values.min(), values.max()
    if vmax - vmin < 1e-10:
        vmin, vmax = -1, 1
    
    # Plot with color mapping
    scatter = ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                        c=values, cmap='RdBu', s=150,
                        vmin=vmin, vmax=vmax,
                        alpha=0.8, edgecolors='k', linewidth=0.5)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.5, pad=0.1)
    cbar.set_label(f'Y_{{{l},{m}}}', rotation=0, labelpad=15)
    
    # Set title and labels
    ax.set_title(f'Y$_{{{l},{m}}}$: {description}', fontsize=12)
    ax.set_box_aspect([1,1,1])
    
    # Clean up axes
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

plt.suptitle('Spherical Harmonics and Their Biological Interpretations', fontsize=16)
plt.tight_layout()
plt.show()

### 4.2 Interactive Harmonic Explorer

In [ ]:
# Create interactive harmonic explorer
cube_vertices = np.array(platonic_solids['cube'].vertices)
explorer = harmonic_viz.create_interactive_explorer(cube_vertices)
display(explorer)

## 5. Biological Applications: Bacteria as Deformed Polytopes

### 5.1 Visualizing Bacterial Deformations

Many biological structures can be modeled as deformed polytopes:
- **Bacteria**: Elongated and bent octahedra
- **Organelles**: Ellipsoidal deformations with internal structure
- **Viral capsids**: Icosahedral symmetry with protein subunits

In [ ]:
# Create bacterial deformation sequence
fig = plt.figure(figsize=(15, 5))

# Parameters for deformation sequence
elongations = [1.0, 1.5, 2.0, 2.5]
bendings = [0.0, 0.1, 0.2, 0.3]
titles = ['Rest State', 'Slight Elongation', 'Bacterial Shape', 'Highly Deformed']

for idx, (elongation, bending, title) in enumerate(zip(elongations, bendings, titles)):
    ax = fig.add_subplot(1, 4, idx+1, projection='3d')
    
    # Create deformed shape
    deformed = visualize_bacteria_deformation(
        rest_shape='octahedron',
        elongation=elongation,
        bending=bending,
        thickness_variation=0.1
    )
    
    vertices = np.array(deformed.vertices)
    edges = np.array(deformed.edges)
    
    # Plot vertices
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
               c='lightgreen', s=100, alpha=0.8, edgecolors='darkgreen')
    
    # Plot edges
    for edge in edges:
        points = vertices[edge]
        ax.plot(points[:, 0], points[:, 1], points[:, 2],
               'darkgreen', alpha=0.6, linewidth=2)
    
    ax.set_title(title, fontsize=12)
    ax.set_box_aspect([1,1,1])
    ax.set_xlim([-3, 3])
    ax.set_ylim([-1.5, 1.5])
    ax.set_zlim([-1.5, 1.5])
    
    # Remove ticks for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

plt.suptitle('Bacterial Deformation Sequence: From Octahedron to Rod Shape', fontsize=14)
plt.tight_layout()
plt.show()

### 5.2 Harmonic Analysis of Biological Shapes

In [ ]:
# Create biological pattern visualizations
bio_patterns = BiologicalHarmonicPatterns()

# Get patterns
patterns = {
    'Bacteria': bio_patterns.bacteria_elongation_pattern(),
    'Organelle': bio_patterns.organelle_pattern(),
    'Viral Capsid': bio_patterns.viral_capsid_pattern()
}

# Use octahedron as base shape
base_vertices = np.array(platonic_solids['octahedron'].vertices)

# Visualize patterns
fig = plt.figure(figsize=(15, 5))

for idx, (name, pattern) in enumerate(patterns.items()):
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    
    # Synthesize shape from harmonics
    total_field = np.zeros(len(base_vertices), dtype=complex)
    for (l, m), coeff in pattern.items():
        Y_lm = harmonic_viz.compute_spherical_harmonic(l, m, base_vertices)
        total_field += coeff * Y_lm
    
    # Use magnitude to scale vertices
    magnitudes = np.abs(total_field)
    scaled_vertices = base_vertices * magnitudes[:, np.newaxis]
    
    # Color by phase
    phases = np.angle(total_field)
    colors = plt.cm.hsv((phases + np.pi) / (2 * np.pi))
    
    # Plot
    scatter = ax.scatter(scaled_vertices[:, 0], scaled_vertices[:, 1], scaled_vertices[:, 2],
                        c=colors, s=200, alpha=0.8, edgecolors='k', linewidth=1)
    
    ax.set_title(f'{name} Pattern', fontsize=14)
    ax.set_box_aspect([1,1,1])
    
    # Add pattern description
    pattern_desc = f"Dominant modes:\n"
    for i, ((l, m), coeff) in enumerate(list(pattern.items())[:3]):
        pattern_desc += f"Y_{{{l},{m}}}: {abs(coeff):.1f}\n"
    
    ax.text2D(0.02, 0.98, pattern_desc, transform=ax.transAxes,
              fontsize=10, verticalalignment='top',
              bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 6. Stereographic Projections

### 6.1 Understanding Stereographic Projection

Stereographic projection maps points on a sphere to a plane. It:
- Preserves angles (conformal)
- Maps circles to circles (or lines)
- Is used in crystallography, complex analysis, and topology

In [ ]:
# Create stereographic projection viewer
stereo_viewer = StereographicProjectionViewer(fig_size=(12, 6))

# Project different polytopes
polytopes_to_project = ['tetrahedron', 'cube', 'octahedron']

for name in polytopes_to_project:
    polytope_vis = vis_polytopes[name]
    stereo_viewer.visualize_projection(polytope_vis, show_sphere=True)
    plt.suptitle(f'Stereographic Projection of {name.capitalize()}', fontsize=16)
    plt.show()

## 7. Polytope Signatures and Comparison

### 7.1 Computing Harmonic Signatures

Each polytope has a unique "harmonic signature" - its decomposition into spherical harmonics. This signature can be used for:
- Shape classification
- Symmetry detection
- Biological structure identification

In [ ]:
# Create signature viewer
sig_viewer = PolytopeSignatureViewer(fig_size=(12, 8))

# Compare signatures of basic polytopes
polytopes_to_compare = {
    name: vis for name, vis in vis_polytopes.items() 
    if name in ['tetrahedron', 'cube', 'octahedron']
}

# Add a deformed bacteria shape
bacteria_shape = visualize_bacteria_deformation(
    rest_shape='octahedron',
    elongation=2.0,
    bending=0.2
)
polytopes_to_compare['bacteria'] = bacteria_shape

# Visualize and compare signatures
sig_viewer.visualize_signatures(polytopes_to_compare)

## 8. Polytope Tilings

### 8.1 Cubic Tiling of Space

Some polytopes can tile 3D space without gaps:
- **Cubic tiling**: Simple cubic lattice
- **Truncated octahedral tiling**: Space-filling with truncated octahedra
- **Quasicrystalline tilings**: Aperiodic tilings with icosahedral symmetry

In [ ]:
# Create small cubic tiling
cubic_tiles = create_tiling_visualization(tiling_type='cubic', extent=2)

# Visualize tiling
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot first few tiles
for tile in cubic_tiles[:8]:  # Limit for clarity
    vertices = np.array(tile.vertices)
    edges = np.array(tile.edges)
    
    # Plot edges
    for edge in edges:
        points = vertices[edge]
        ax.plot(points[:, 0], points[:, 1], points[:, 2],
               'b-', alpha=0.6, linewidth=1)
    
    # Plot vertices
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
               c='red', s=30, alpha=0.8)

ax.set_title('Cubic Tiling of 3D Space', fontsize=16)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_box_aspect([1,1,1])

# Set viewing angle
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

## 9. Advanced Visualization: Harmonic Decomposition

### 9.1 Decomposing Complex Shapes

In [ ]:
# Create an elongated ellipsoidal shape (like a bacterium)
theta = np.linspace(0, np.pi, 20)
phi = np.linspace(0, 2*np.pi, 40)
theta_grid, phi_grid = np.meshgrid(theta, phi)

# Elongated ellipsoid parameters
a, b, c = 2.0, 0.7, 0.7  # Semi-axes

# Generate vertices
x = a * np.sin(theta_grid.flatten()) * np.cos(phi_grid.flatten())
y = b * np.sin(theta_grid.flatten()) * np.sin(phi_grid.flatten())
z = c * np.cos(theta_grid.flatten())
ellipsoid_vertices = np.stack([x, y, z], axis=-1)

# Perform harmonic decomposition
decomposition = harmonic_viz.decompose_shape(ellipsoid_vertices)

# Display decomposition results
print("Shape Analysis Results:")
print(f"Biological signature: {decomposition.biological_signature}")
print(f"\nDominant harmonic modes:")
for i, (l, m) in enumerate(decomposition.dominant_modes[:5]):
    coeff = decomposition.coefficients[(l, m)]
    print(f"  {i+1}. Y_{{{l},{m}}}: magnitude = {abs(coeff):.3f}, phase = {np.angle(coeff):.3f}")

# Visualize decomposition
fig = harmonic_viz.visualize_decomposition(decomposition, top_k=6)
plt.show()

## 10. Summary and Key Takeaways

In this notebook, we've explored:

1. **Platonic Solids**: The fundamental regular polytopes in 3D
2. **Interactive Visualization**: Using pythreejs for 3D manipulation
3. **Spherical Harmonics**: Mathematical basis for shape analysis
4. **Biological Applications**: Modeling bacteria and organelles as deformed polytopes
5. **Stereographic Projections**: Mapping 3D structures to 2D
6. **Harmonic Signatures**: Unique shape fingerprints
7. **Polytope Tilings**: Space-filling patterns

### Key Insights:

- **Shape-Function Relationship**: Biological structures often adopt polytope-like forms for optimal function
- **Harmonic Analysis**: Provides a natural way to quantify and compare 3D shapes
- **Symmetry**: Platonic solids embody fundamental symmetries found throughout nature
- **Deformation**: Simple polytopes can model complex biological shapes through systematic deformations

### Next Steps:

- Explore the 24-cell and higher-dimensional polytopes
- Apply these techniques to real microscopy data
- Investigate the connection between harmonic signatures and biological function
- Use polytope matching for bacterial segmentation tasks

## 11. Interactive Playground

Use this section to experiment with different parameters and visualizations:

In [ ]:
# Interactive parameter exploration
@widgets.interact(
    polytope=['tetrahedron', 'cube', 'octahedron'],
    l=(0, 6, 1),
    m=(-6, 6, 1),
    show_phase=True
)
def explore_harmonics(polytope='octahedron', l=2, m=0, show_phase=True):
    # Get polytope vertices
    vertices = np.array(platonic_solids[polytope].vertices)
    
    # Validate m range
    m = max(-l, min(l, m))
    
    # Create visualization
    fig = harmonic_viz.visualize_single_harmonic(
        vertices, l=l, m=m, show_phase=show_phase
    )
    plt.show()

In [ ]:
# Interactive bacterial deformation
@widgets.interact(
    elongation=(1.0, 3.0, 0.1),
    bending=(0.0, 0.5, 0.05),
    thickness_variation=(0.0, 0.3, 0.05)
)
def explore_deformations(elongation=2.0, bending=0.2, thickness_variation=0.1):
    # Create deformed shape
    deformed = visualize_bacteria_deformation(
        rest_shape='octahedron',
        elongation=elongation,
        bending=bending,
        thickness_variation=thickness_variation
    )
    
    # Visualize
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    vertices = np.array(deformed.vertices)
    edges = np.array(deformed.edges)
    
    # Plot
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
               c='lightgreen', s=150, alpha=0.8, edgecolors='darkgreen')
    
    for edge in edges:
        points = vertices[edge]
        ax.plot(points[:, 0], points[:, 1], points[:, 2],
               'darkgreen', alpha=0.6, linewidth=2)
    
    ax.set_title(f'Elongation: {elongation:.1f}, Bending: {bending:.2f}', fontsize=14)
    ax.set_box_aspect([1,1,1])
    ax.set_xlim([-3, 3])
    ax.set_ylim([-2, 2])
    ax.set_zlim([-2, 2])
    
    plt.tight_layout()
    plt.show()